In [3]:
from decouple import Config, RepositoryEnv
from datetime import datetime, date, timedelta

from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame, TimeFrameUnit

from alpaca.trading.client import TradingClient
from alpaca.trading.requests import  MarketOrderRequest
from alpaca.trading.enums import OrderSide, TimeInForce

In [4]:
config = Config(RepositoryEnv(r"D:\1_COde\All\1量化\CQF\algo trading\.env"))

# 之后就可以像之前一样用
api_key = config("ALPACA_KEY")
secret_key = config("ALPACA_SECRET")

In [5]:
# instatiate the trading client
api = TradingClient(api_key, secret_key, raw_data=True) # raw_data = True returns python dict and not convert the output into object


In [6]:
# clock
clock = api.get_clock()
# get the account details
account = api.get_account()
# get a list of all of our positions.
portfolio = api.get_all_positions()

In [7]:
# Trading App
class App():

    def run(self):
        lookback = 20
        quantity = 10
        stock = "AAPL"

        # retrieve data
        stock_client = StockHistoricalDataClient(api_key, secret_key)
        end_date = datetime.today() #- timedelta(days=1)
        start_date = end_date - timedelta(days=1)

        if clock.get('is_open') == True:
            
            # get data
            data = stock_client.get_stock_bars(
                StockBarsRequest(
                symbol_or_symbols = ["AAPL"],
                start = start_date,
                end = end_date,
                timeframe = TimeFrame(1, TimeFrameUnit.Minute))).df

            # signal logic
            h = data['high']
            l = data['low']

            portfolio = api.get_all_positions()
            if bool(portfolio) == False:
                if h[-1] > h[-5]:
                    api.submit_order(MarketOrderRequest(symbol = stock, qty = 10, side = OrderSide.BUY, time_in_force = TimeInForce.DAY))
                    print(f"Bought {quantity} share(s) of {stock}")
                elif l[-1] < l[-5]:
                    # api.submit_order(MarketOrderRequest(symbol = stock, qty = 10, side = OrderSide.SELL, time_in_force = TimeInForce.DAY))
                    api.close_position(symbol_or_asset_id=stock)
                    print(f"Sold {quantity} share(s) of {stock}")
            else:
                balance_change = float(account.equity) - float(account.last_equity)
                print("Market Closed")
                print(f"Today's portfolio balance change is : {balance_change}")

In [7]:
# Trading App
class App():

    def run(self):
        lookback = 20
        quantity = 10
        stock = "AAPL"

        # ------- 支撑/阻力参数 -------
        n_window = 50        # 支撑/阻力窗口（分钟）
        buffer   = 0.0015    # 突破缓冲（0.15%）
        # ---------------------------

        # retrieve data
        stock_client = StockHistoricalDataClient(api_key, secret_key)
        end_date = datetime.today()
        start_date = end_date - timedelta(days=1)

        if clock.get('is_open') == True:

            # get data
            data = stock_client.get_stock_bars(
                StockBarsRequest(
                    symbol_or_symbols=[stock],
                    start=start_date,
                    end=end_date,
                    timeframe=TimeFrame(1, TimeFrameUnit.Minute)
                )
            ).df

            # Alpaca 可能返回 MultiIndex (symbol, time)，取出该股票的切片
            import pandas as pd
            if isinstance(data.index, pd.MultiIndex):
                data = data.xs(stock)

            data = data.sort_index()

            # —— Donchian 支撑/阻力（不含当前bar：用 shift(1)）——
            h = data['high']
            l = data['low']
            c = data['close']

            # 历史长度不足直接返回（避免 NaN）
            if len(c) < n_window + 2:
                print("Not enough data for S/R yet.")
                return

            resistance = h.rolling(n_window).max().shift(1)  # 阻力：过去N根最高
            support    = l.rolling(n_window).min().shift(1)  # 支撑：过去N根最低

            r = resistance.iloc[-1]
            s = support.iloc[-1]
            p_now  = c.iloc[-1]
            p_prev = c.iloc[-2]

            if pd.isna(r) or pd.isna(s):
                print("Warming up S/R windows...")
                return

            # 当前是否有持仓
            portfolio = api.get_all_positions()
            has_pos = bool(portfolio)

            if not has_pos:
                # —— 上破阻力 → 买入（加缓冲防假突破）
                if (p_prev <= r * (1 + buffer)) and (p_now > r * (1 + buffer)):
                    api.submit_order(MarketOrderRequest(
                        symbol=stock, qty=quantity, side=OrderSide.BUY, time_in_force=TimeInForce.DAY
                    ))
                    print(f"[BUY] Breakout above R={r:.2f} @ {p_now:.2f}")

                # —— 下破支撑 → 平仓/做空（这里默认平仓；若已开通卖空可改为 SELL 开空）——
                elif (p_prev >= s * (1 - buffer)) and (p_now < s * (1 - buffer)):
                    try:
                        api.close_position(symbol_or_asset_id=stock)
                        print(f"[FLAT] Breakdown below S={s:.2f} @ {p_now:.2f}")
                    except Exception as e:
                        print("No position to close or error:", e)

            else:
                # 已有仓位：简单的反向信号退出（跌破支撑则平）
                if (p_prev >= s * (1 - buffer)) and (p_now < s * (1 - buffer)):
                    try:
                        api.close_position(symbol_or_asset_id=stock)
                        print(f"[EXIT] Breakdown below S={s:.2f} @ {p_now:.2f}")
                    except Exception as e:
                        print("Close error:", e)

                # 也可以加触达阻力的止盈（可选）：
                # if p_now >= r * (1 - buffer/2):
                #     api.close_position(symbol_or_asset_id=stock)
                #     print(f"[TP] Near resistance {r:.2f}, close @ {p_now:.2f}")

        else:
            balance_change = float(account.equity) - float(account.last_equity)
            print("Market Closed")
            print(f"Today's portfolio balance change is : {balance_change}")


In [8]:
if __name__ == "__main__":  
    app = App()
    while clock.get('is_open'):
        app.run()
    else:
        print(f"Market is closed, Next open at {clock.get('next_open')}")
        balance_change = float(account.get('equity')) - float(account.get('last_equity'))
        print(f"Today's portfolio balance change is : {balance_change:.2f}")     

Market is closed, Next open at 2025-09-15T09:30:00-04:00
Today's portfolio balance change is : 0.00
